
# Do końcowego projektu

- Dodac spoznienia/przyjscia wczesniej w umawianym

- csv z danymi o id_Pacjenta, id_gabinetu, arrival_time, service_start_time, czas_oczekiwania, service_end_time, czas_

- procesowanie danych

- lekarze przerwa od pracy


# Do zrobienia "Sanity checku"

- naprawić żeby klienci nie wchodzili do gabinetu {service_time} przed zamknięciem **chyba zrobione**

- dlugość kolejki zapisywana

In [205]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import simpy
import seaborn
import matplotlib.pyplot as plt

In [206]:
class Pacjent:
    def __init__(self, id):
        self.id = id
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None


In [207]:
class Gabinet:
    def __init__(self, id, env):
        self.id = id
        self.env = env
        self.patients_served = 0

In [208]:
class Clinic:
    def __init__(self, env, number_of_rooms, service_time, queue_array=[], lambda_per_hour=12/60, seed=None, sim_time=480):
        self.curr_patient_id = 1
        self.env = env
        self.service_time = service_time
        self.lambda_per_hour = lambda_per_hour
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1, env=self.env) for i in range(number_of_rooms)]
        self.rooms = simpy.Store(self.env, capacity=number_of_rooms)
        self.sim_time = sim_time
        self.queue_array = queue_array
        for room in self.list_rooms:
            self.rooms.put(room)
        self.processed_patients = []

    def time_between_new_patients(self):
        if self.seed:
            np.random.seed(self.seed)
        return np.random.exponential(1/self.lambda_per_hour)
    
    def generate_patients(self):
        while True:
            if self.env.now >= self.sim_time-self.service_time: # Pacjenci nie przychodzą przed {service_time} zamknięciem
                break
            patient = Pacjent(id=self.curr_patient_id)
            patient.arrival_time = self.env.now
            print(f"Czas {np.trunc(self.env.now)}: Pacjent {patient.id} przybył do kliniki")
            self.env.process(self.serve_patient(patient))
            self.curr_patient_id += 1
            yield self.env.timeout(self.time_between_new_patients())

    def serve_patient(self, patient): 
        room = yield self.rooms.get()
        self.queue_array.append(patient.id)
        if self.env.now >= self.sim_time-self.service_time: # Nie zdążyło obsłużyć pacjenta przed zamknięciem
            print(f"Czas {np.trunc(self.env.now)}: Pacjent {patient.id} - nie zdążył zostać obsłużony przed zamknięciem kliniki")
            yield self.rooms.put(room)
            return
        patient.service_start_time = self.env.now
        print(f"Czas {np.trunc(self.env.now)}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
        self.queue_array.remove(patient.id)
        yield self.env.timeout(self.service_time)

        patient.service_end_time = self.env.now
        room.patients_served += 1
        print(f"Czas {np.trunc(self.env.now)}: Pacjent {patient.id} wychodzi z gabinetu {room.id}, czekal od {np.trunc(patient.arrival_time)} do {np.trunc(patient.service_start_time)}")
        self.processed_patients.append(patient)
        yield self.rooms.put(room)

    def run(self):
        self.env.process(self.generate_patients())
        env.run(until=self.sim_time)

    def stats(self):
        def patient_bar_plot():
            patients_served_ls = []
            id_ls = []
            for room in self.list_rooms:
                patients_served_ls.append(room.patients_served)
                id_ls.append(str(room.id))
            fig = seaborn.barplot(x=id_ls, y=patients_served_ls)
            fig.set_xlabel('Gabinet')
            fig.set_ylabel('Ilosc Pacjentow')
            plt.savefig("patients_served_noshow.png")
            plt.show()

        patient_bar_plot()

In [209]:
#bez umawiania
env = simpy.Environment()

In [210]:
clinic = Clinic(env, number_of_rooms=3, service_time=15)

In [211]:
clinic.run()

Czas 0: Pacjent 1 przybył do kliniki
Czas 0: Pacjent 1 wchodzi do gabinetu 1 
Czas 5.0: Pacjent 2 przybył do kliniki
Czas 5.0: Pacjent 2 wchodzi do gabinetu 2 
Czas 6.0: Pacjent 3 przybył do kliniki
Czas 6.0: Pacjent 3 wchodzi do gabinetu 3 
Czas 13.0: Pacjent 4 przybył do kliniki
Czas 15: Pacjent 1 wychodzi z gabinetu 1, czekal od 0 do 0
Czas 15: Pacjent 4 wchodzi do gabinetu 1 
Czas 16.0: Pacjent 5 przybył do kliniki
Czas 20.0: Pacjent 6 przybył do kliniki
Czas 20.0: Pacjent 2 wychodzi z gabinetu 2, czekal od 5.0 do 5.0
Czas 20.0: Pacjent 5 wchodzi do gabinetu 2 
Czas 21.0: Pacjent 3 wychodzi z gabinetu 3, czekal od 6.0 do 6.0
Czas 21.0: Pacjent 6 wchodzi do gabinetu 3 
Czas 28.0: Pacjent 7 przybył do kliniki
Czas 30: Pacjent 4 wychodzi z gabinetu 1, czekal od 13.0 do 15
Czas 30: Pacjent 7 wchodzi do gabinetu 1 
Czas 34.0: Pacjent 8 przybył do kliniki
Czas 35.0: Pacjent 5 wychodzi z gabinetu 2, czekal od 16.0 do 20.0
Czas 35.0: Pacjent 8 wchodzi do gabinetu 2 
Czas 36.0: Pacjent 6 wy

In [212]:
clinic.queue_array

[91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121]

In [213]:
clinic.stats()